In [27]:
#install dependencies pandas openpyxl asana itertools re and requests
import pandas as pd
from openpyxl import load_workbook
import asana  
from asana.rest import ApiException  
import os
import itertools
import re
import requests
import tkinter as tk
from tkinter import filedialog
#Dict for all strains and primer sets
substanceStrainPrimer = { #dictonary of keys 
                         }
#Dict for primer requierd for cetrain feild trials in 2023
substanceStrainPrimerSummer2023 = {#dictonary of keys  
  
}


In [28]:
#location of file in your coputer and reading the sheet name 
##############################################################Change the pah in the gDNA_File = 'Path to your file'
gDNA_File = ''
df = pd.read_excel(gDNA_File, sheet_name = "gdna_samples")

In [29]:
#Getting number of rows in the Data Frame
num_rows = df.shape [0]
#################################################################EDIT ME if you need to change the type of measuremnts
measurement_type = ['qPCR, NovaSeq'] * num_rows
df['Measurements'] = measurement_type
#print(num_rows) to see tyhe amount of rows in the file

In [30]:
#Working with primer columns
allSubstanceStrains = df['substance_strains'].drop_duplicates().tolist()
# Function to split the elements in the list
def split_elements(allSubstanceStrains):
    result_allSubstanceStrains = []
    for item in allSubstanceStrains:
        if isinstance(item, str) and ',' in item:
            sub_items = item.split(', ')
            result_allSubstanceStrains.extend(sub_items)
        else:
            result_allSubstanceStrains.append(item)
    return result_allSubstanceStrains

#Pulling Primers
allSubstanceStrains = split_elements(allSubstanceStrains)
substanceStrainsPrimerMatch = {item: substanceStrainPrimer.get(item, "Not Found") for item in allSubstanceStrains}
substanceStrainsPrimerMatchValues = list(substanceStrainsPrimerMatch.values())
substanceStrainsPrimerMatchValuesString = ', '.join(str(value) for value in substanceStrainsPrimerMatchValues)
#Roemoving extra naming conventions like []''
substanceStrainsPrimerMatchClean = {value for value in substanceStrainsPrimerMatchValuesString.split(', ') if value not in ('Not Found', '', '[]', '""')}
#Removing extra characters
pattern = r'[,a-zA-Z0-9_]'
substanceStrainsPrimerMatchFiltered = []
for item in substanceStrainsPrimerMatchClean:
    # Use re.findall() to find all matches in the list
    filtered_item = ''.join(re.findall(pattern, item))
    substanceStrainsPrimerMatchFiltered.append(filtered_item)
substanceStrainsPrimerMatchFiltered = substanceStrainsPrimerMatchFiltered[:len(df)]
#turning list into string
substanceStrainsPrimerMatchString = ', '.join(substanceStrainsPrimerMatchFiltered)
print(substanceStrainsPrimerMatchString)
primer_set_strains = [substanceStrainsPrimerMatchString] * num_rows
df['primer_set_strains'] = primer_set_strains

F2702_R2703_P2790, F6824_R6826_P6825, F11809_R11810_P13823, F10809_R10810_P10811


In [31]:
#Warning code to find if a primer or substance has no match
#Finding primers and substances 
def find_primer_matches(allSubstanceStrains, substanceStrainPrimer):
    missing_primers = []
    for allSubstanceStrains in allSubstanceStrains:
        found_match = False
        for substanceStrainPrimer in substanceStrainPrimer:
            if allSubstanceStrains in substanceStrainPrimer:
                found_match = True
                break
        if not found_match:
            missing_primers.append(allSubstanceStrains)
    return missing_primers
#If statement to tell user warning messages
def main():
    allSubstanceStrains = []
    substanceStrainPrimer = []
    missing_primers = find_primer_matches(allSubstanceStrains, substanceStrainPrimer)
    if missing_primers:
        print("Warning: The following primers have no matches in the substace_strains:")
        for allSubstanceStrains in missing_primers:
            print(allSubstanceStrains)
    else:
        print("All primers have at least one match in the substace_strains.")

if __name__ == "__main__":
    main()


All primers have at least one match in the substace_strains.


In [32]:
#reading the gDNA File to append gDNA Quantifdications tab
df2 = pd.read_excel(gDNA_File, sheet_name = "gdna_quantifications")
#Replacing file name with _qPCR_Request.xlsx
gDNA_File_w_qPCRPrimers = gDNA_File.replace('.xlsx', '_qPCR_request.xlsx')
# Create an Excel writer object with the openpyxl engine
with pd.ExcelWriter(gDNA_File_w_qPCRPrimers, engine='openpyxl') as writer:
    # Write each DataFrame to a separate sheet in the Excel file
    df.to_excel(writer, index=False, sheet_name='gdna_samples')
    df2.to_excel(writer, index=False, sheet_name='gdna_quantifications')


In [ ]:
#Taking File name to use for ASANA
file_name = os.path.basename(gDNA_File)
absolute_path = os.path.abspath(os.path.join(gDNA_File))
#Removing copy and "_" from name
def remove_copy_and_underscore(file_name):
    base_name, extension = os.path.splitext(file_name)
    if base_name.endswith("_copy"): 
        base_name = base_name[:-len("_copy")]
        base_name = base_name.replace(" ", "")
    new_name = base_name.replace("_", " ") + extension
    return new_name
#Calling function above
qPCRRequestname = remove_copy_and_underscore(file_name)
print(qPCRRequestname)
print("File Path:", absolute_path)

In [69]:
#The ASANA personal Access Token can be found after making an ASANA API app
#https://app.asana.com/0/my-apps
asana_personal_access_token = ''
#Project GID 
project_gid = ''
project_name = 'qPCR pipeline'

base_url = 'https://app.asana.com/api/1.0'
headers = {
    'Authorization': f'Bearer {asana_personal_access_token}'
}
configuration = asana.Configuration()
configuration.access_token = ''

def create_asana_task_with_details(project_gid, task_name, assignee_gid, due_date, followers_gids=[]):
    client = asana.Configuration.access_token
# Suppress deprecation warnings
    client.headers['Asana-Disable'] = 'new_user_task_lists,new_goal_memberships'
    data = {
        'projects': [project_gid],
        'name': task_name,
        'assignee': assignee_gid,
        'due_on': due_date,
        'followers': followers_gids,
    }
    result = client.tasks.create(data)


In [70]:
#Function to attach file to path 
def attach_file_to_task(task_gid, absolute_path):
    client = asana.Configuration.get_basic_auth_token(asana_personal_access_token)
    # Suppress deprecation warnings
    client.headers['Asana-Disable'] = 'new_user_task_lists,new_goal_memberships'
    with open(absolute_path, 'rb') as file:
        file_content = file.read()
        result = client.attachments.create_on_task(task_gid, file_content, file_name= absolute_path)
    
    if result['errors']:
        print(f"Error attaching file: {result['errors']}")
        return False

    print("File attached to the task.")
    return True

# Function to move the task to a section
def move_task_to_section(task_gid, section_gid):
    client = asana.Con(asana_personal_access_token)
    result = client.tasks.add_project(task_gid, project={'gid': section_gid})
    if result['errors']:
        print(f"Error moving task to section: {result['errors']}")
        return False
    print("Task moved to the specified section.")
    return True


In [71]:
# Example usage:
personal_access_token = asana_personal_access_token
task_name = qPCRRequestname
assignee_gid = ''
due_date = '2023-09-15'  # Replace with the due date in ISO 8601 format (e.g., 'YYYY-MM-DD')
followers_gids = []
file_path = absolute_path  # Replace with the path to the file you want to attach (or set to None if no attachment needed)
project_gid = ''  # Replace with the GID of the project where you want to create the task
section_gid = 'Requested (top=highest priority)' 

In [ ]:
#################TROUBLESHOOTINGto read error message!##########################################################
# Create the task
url = f'{base_url}/tasks'
data = {
        'projects': [project_gid],  # Directly use the provided project GID
        'name': task_name,
        'assignee': assignee_gid,
        'due_on': due_date,
        'followers': followers_gids,
    }
response = requests.post(url, headers=headers, json=data)
if response.status_code != 201:
    error_message = response.json().get('message', 'Unknown Error')
    print(f"Error creating task: {response.status_code} - {error_message}")
    print("Response content:", response.json())

Error creating task: 400 - Unknown Error
Response content: {'errors': [{'message': 'You should specify one of workspace, parent, projects', 'help': 'For more information on API status codes and how to handle them, read the docs on errors: https://developers.asana.com/docs/errors'}]}
